<a href="https://colab.research.google.com/drive/1PZrItIUNQdS3sAyTz18sB60os1fWUc8Y#scrollTo=P7pmhsHW6ErY" 
target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block alert-info">

<h1 style="font-family:verdana;"> Description:</h1> 

<ul>
<li><p style="font-family:verdana;">
In this notebook, we are going to predict the lead of Credit Card, using various features such as Gender, Age, Region_Code, Occupation, Channel_Code, Vintage, Credit_Product, Avg_Account_Balance, Is_Active.
</p></li>
    
<li><p style="font-family:verdana;">
The dataset we are going to use is the Credit Card Lead Prediction dataset from Analytics Vidhya which contains about 245725 rows and 10 features that can be downloaded <a href="https://datahack.analyticsvidhya.com/contest/job-a-thon">here</a>.
</p></li> 

<li><p style="font-family:verdana;">
The dataset contains the labels which we have to predict and the labels are continuous. So the problem we have is a Supervised Regression type.
</p></li>  
</ul>

</div>

## Step 0: Import libraries and dataset

In [ ]:
# Importing All Required libraries
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn import metrics

In [ ]:
# Importing Datasets of train file and test file

train=pd.read_csv("/content/train_s3TEQDk.csv")
test=pd.read_csv("/content/test_mSzZ8RL.csv")


## Step 1: Descriptive analysis

In [ ]:
 Preview training dataset
train.head()

In [ ]:
# Preview testing dataset
test.head()

In [ ]:
# Training dataset dimensions - (rows, columns)
print('Training data: \nRows: {} Columns: {}'.format(train.shape[0], train.shape[1]))

In [ ]:
# Testing dataset dimensions - (rows, columns)
print('Testing data: \nRows: {} Columns: {}'.format(test.shape[0], test.shape[1]))

In [ ]:
# Features data-type
train.info()

In [ ]:
# Statistical summary
train.describe().T

In [ ]:
# Checking for Null values
round((train.isnull().sum() / train.shape[0]) * 100, 2).astype(str) + ' %'

<div class="alert alert-block alert-info">

<h3 style="font-family:verdana;"> Observations:</h3>

<ul>
    
<li><p style="font-family:verdana;">
The feature 'Credit_Product' contains 31.57% null values, So we want to use label encoding technique to remove null values.
</p>   
    
<li><p style="font-family:verdana;">
Here we observe some of the features has datatype  int and object, So we want to keep all same as int. so here we ise  same technique which is label encoding to put as all int in features.
</p> 
</ul>

</div>

## Step 3: Data preprocessing

### 3.1: Replacing dtype as str in 'Credit_Product' and 'Channel_Code' and 'Is_Active'

In [ ]:
train['Credit_Product']=train['Credit_Product'].astype(str)
train['Channel_Code']=train['Channel_Code'].astype(str)
train['Is_Active']=train['Is_Active'].astype(str)

### 3.2: Feature Encoding

In [ ]:
le = LabelEncoder()
train['Gender']= le.fit_transform(train['Gender'])
train['Occupation']= le.fit_transform(train['Occupation'])
train['Credit_Product']= le.fit_transform(train['Credit_Product'])
train['Vintage']= le.fit_transform(train['Vintage'])
train['Region_Code']= le.fit_transform(train['Region_Code'])
train['Channel_Code']= le.fit_transform(train['Channel_Code'])
train['Age']= le.fit_transform(train['Age'])
train['Is_Active']= le.fit_transform(train['Is_Active'])
train['ID']= le.fit_transform(train['ID'])

In [ ]:
test=pd.read_csv("/content/gdrive/MyDrive/Untitled form (File responses)/test_mSzZ8RL.csv")
 
test['Credit_Product']=test['Credit_Product'].astype(str)
df=test.copy()
lab=LabelEncoder()
df["Gender"]=lab.fit_transform(test["Gender"])
df["Occupation"]=lab.fit_transform(test["Occupation"])
df["Credit_Product"]=lab.fit_transform(test["Credit_Product"])
df["Is_Active"]=lab.fit_transform(test["Is_Active"])
df["Channel_Code"]=lab.fit_transform(test["Channel_Code"])
df["ID"]=lab.fit_transform(test["ID"])
df["Age"]=lab.fit_transform(test["Age"])
df["Region_Code"]=lab.fit_transform(test["Region_Code"])
df["Vintage"]=lab.fit_transform(test["Vintage"])
df["Avg_Account_Balance"]=lab.fit_transform(test["Avg_Account_Balance"])
df.head()
 
 
# target1=df.iloc[:,10:]
input1=df.iloc[:,0:10]
input1

### 3.3: Separating train into X and Y

In [ ]:
target = 'Is_Lead'
df1=train.copy()
X = df1.loc[:, df1.columns!=target]
Y = df1.loc[:, df1.columns==target]

## Step 4: Data Modelling

### 4.1: cat boost Regressor

In [ ]:
from catboost import CatBoostRegressor
best_params = {
            'bagging_temperature': 0.5,
            'depth': 8,
            'iterations': 2000,
            'l2_leaf_reg': 25,
            'learning_rate': 0.0556,
            'sampling_frequency': 'PerTreeLevel',
            'leaf_estimation_method': 'Gradient',
            'random_strength': 0.8,
            'boosting_type': 'Ordered',
            'feature_border_type': 'MaxLogSum',
            'l2_leaf_reg': 50,
            'max_ctr_complexity': 2,
            'fold_len_multiplier': 2
    }


In [ ]:

from catboost import CatBoostRegressor
model = CatBoostRegressor(**best_params)
model.fit(X,Y,verbose=1000)
predictions_catboost = model.predict(input1)

### 4.2: xgb classifier

In [ ]:
import xgboost as xg
from xgboost import XGBClassifier
 
clf2 = xg.XGBClassifier(class_weight='balanced').fit(X_train, Y_train)
Y_Test_Pred = clf2.predict(X_test)


## Step 5: Model Evaluation

In [ ]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print("F1 Score = " ,f1_score(y_actual, y_predicted))
    pass

In [ ]:
def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(Y_test,  y_pred_proba)
    auc = roc_auc_score(Y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
    plt.legend(loc=4)
    plt.show()
    pass

### Saving the model

In [ ]:
from joblib import dump, load
dump(model, 'extra1.joblib')

pre=load("extra1.joblib")
predicted=pre.predict(input1)
ff=pd.DataFrame({"ID":input1["ID"],
                 "Is_Lead"
                 :predicted})
ff.to_csv ('dataframe.csv', index = None, header=True) 

<div class="alert alert-block alert-info">

<h3 style="font-family:verdana;"> Conclusion:</h3>

<ul>
    
<li><p style="font-family:verdana;">
In this project, we tried to build a model using various algorithms such as svm, Decision tree regression, Random forest and XGB regressor and XGB Classifier to get the best possible prediction.
</p></li>     
        
<li><p style="font-family:verdana;">
The hyperparameter tuned catboost regressor gives us the best roc_aucscore and r2 score for this problem.
</p></li>    

   

</ul>

</div>